In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.284196,0.300568,332.705560,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,189.804
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.277973,0.301710,332.945945,2.029477e+08,1.061597e+07,17.592548,1.852326e+08,0.776975,174.713
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.271750,0.302645,335.222130,2.031806e+08,1.062582e+07,17.595559,1.852864e+08,0.777079,180.801
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.265527,0.303413,334.785737,2.034136e+08,1.063567e+07,17.598570,1.853401e+08,0.777183,180.172
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.259304,0.304034,335.050185,2.036466e+08,1.064551e+07,17.601581,1.853939e+08,0.777287,185.597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549782,616.073841,NaN,NaN,NaN,NaN,NaN,412.177
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546864,614.423079,NaN,NaN,NaN,NaN,NaN,337.056
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544541,613.772150,NaN,NaN,NaN,NaN,NaN,340.018
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540629,614.761140,NaN,NaN,NaN,NaN,NaN,383.477


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Paraná - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.130907,0.732054,0.692024,1.202413,-0.413954,0.937022,1.076355,1.563435
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.076074,0.695547,0.722303,1.204670,-0.429516,0.934746,1.074731,1.542097
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.021241,0.667742,0.729289,1.206927,-0.445078,0.932469,1.073107,1.520760
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.033592,0.634897,0.709776,1.209183,-0.460640,0.930193,1.071484,1.499423


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
157    280.736
158    352.332
159    280.856
160    319.397
161    295.932
Name: Paraná - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.143258,0.568952,0.770222,1.213697,-0.491764,0.925640,1.068236,1.456748
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.198091,0.552768,0.790292,1.215954,-0.507326,0.923363,1.066613,1.435411
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.252924,0.538720,0.804647,1.218210,-0.522888,0.921087,1.064989,1.414074
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-0.307757,0.529116,0.820506,1.220467,-0.538450,0.918810,1.063365,1.392736
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-0.362590,0.520829,0.835357,1.222724,-0.554011,0.916534,1.061742,1.371399
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-0.417423,0.523857,0.853711,1.224981,-0.569573,0.914257,1.060118,1.350062
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-0.472256,0.532033,0.846173,1.227237,-0.585135,0.911981,1.058494,1.328724
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-0.493807,0.541993,0.860049,1.224389,-0.583336,0.905995,1.051683,1.305447
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-0.515358,0.551845,0.885866,1.221541,-0.581537,0.900008,1.044872,1.282170
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-0.536909,0.565999,0.879636,1.218693,-0.579738,0.894022,1.038060,1.258893


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    354.558
163    360.106
164    350.123
165    288.254
166    301.687
167    268.901
168    274.431
169    298.219
170    328.955
171    335.039
172    260.723
173    366.929
174    353.426
175    344.937
176    302.082
177    294.467
178    336.029
179    278.117
180    323.005
181    291.453
182    300.203
183    317.058
184    314.142
185    301.878
186    358.797
187    356.169
188    316.188
189    346.214
190    323.401
191    238.466
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1633467630, 1675636196, 3809596830, 1616354214, 3912200630, 1217486749, 3813094631, 3609619645, 36530721, 2380056462, 1038529129, 1238414210, 297138764, 3210089238, 1488807308, 300420039, 3988177086, 2575384715, 647476135, 800966040, 3810540045, 206373658, 1909072379, 712135588, 921838056]


Step: 0 ___________________________________________
val_loss: 408.8321228027344
winner_seed: 1633467630


Step: 1 ___________________________________________


2023-10-14 17:54:55.967025: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-14 17:54:56.006967: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 462.48223876953125


Step: 2 ___________________________________________
val_loss: 410.6127014160156


Step: 3 ___________________________________________
val_loss: 499.3442077636719


Step: 4 ___________________________________________
val_loss: 1334.7939453125


Step: 5 ___________________________________________
val_loss: 480.9276123046875


Step: 6 ___________________________________________
val_loss: 465.77288818359375


Step: 7 ___________________________________________
val_loss: 542.5612182617188


Step: 8 ___________________________________________
val_loss: 528.1002197265625


Step: 9 ___________________________________________
val_loss: 444.2518310546875


Step: 10 ___________________________________________
val_loss: 488.12286376953125


Step: 11 ___________________________________________
val_loss: 499.8350524902344


Step: 12 ___________________________________________


2023-10-14 18:01:43.844521: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 586.665283203125


Step: 13 ___________________________________________
val_loss: 495.22332763671875


Step: 14 ___________________________________________
val_loss: 459.454345703125


Step: 15 ___________________________________________
val_loss: 435.5867919921875


Step: 16 ___________________________________________
val_loss: 446.0794372558594


Step: 17 ___________________________________________
val_loss: 551.1141967773438


Step: 18 ___________________________________________
val_loss: 580.7311401367188


Step: 19 ___________________________________________
val_loss: 504.1854553222656


Step: 20 ___________________________________________
val_loss: 504.8590393066406


Step: 21 ___________________________________________
val_loss: 541.8212280273438


Step: 22 ___________________________________________
val_loss: 446.889892578125


Step: 23 ___________________________________________
val_loss: 525.7512817382812


Step: 24 ___________________________________________
val_lo

In [18]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 24ms/step - loss: 87894.1484 - val_loss: 108002.1094
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 68085.7812 - val_loss: 44666.4062
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 36365.4766 - val_loss: 79111.8047
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 67545.5938 - val_loss: 58297.9844
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 43658.4570 - val_loss: 34464.5273
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 42621.1758 - val_loss: 32112.8691
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 21456.6836 - val_loss: 6417.7412
Epoch 8/10000
5/5 [==============================] - 0s 6ms/step - loss: 12491.0537 - val_loss: 14838.9971
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 12106.9736 - val_loss: 7461.1875
Epoch 10/10000
5/5 [=================

Epoch 79/10000
5/5 [==============================] - 0s 5ms/step - loss: 795.2693 - val_loss: 724.8357
Epoch 80/10000
5/5 [==============================] - 0s 5ms/step - loss: 680.4286 - val_loss: 1317.9315
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 588.4490 - val_loss: 1186.5858
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 541.9902 - val_loss: 1154.7334
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 611.9245 - val_loss: 754.1584
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 559.5034 - val_loss: 787.6329
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 835.6395 - val_loss: 1043.9237
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 674.1232 - val_loss: 1217.0898
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 579.6667 - val_loss: 1577.1969
Epoch 88/10000
5/5 [==============================] - 0s 7

5/5 [==============================] - 0s 5ms/step - loss: 756.3920 - val_loss: 957.1919
Epoch 157/10000
5/5 [==============================] - 0s 5ms/step - loss: 598.0702 - val_loss: 1071.9407
Epoch 158/10000
5/5 [==============================] - 0s 5ms/step - loss: 645.1237 - val_loss: 831.5080
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 651.2945 - val_loss: 739.2527
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 512.5764 - val_loss: 778.6285
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 479.7567 - val_loss: 1058.2808
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 656.7845 - val_loss: 1235.2994
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 940.2617 - val_loss: 1540.2517
Epoch 164/10000
5/5 [==============================] - 0s 6ms/step - loss: 976.1498 - val_loss: 927.7648
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step 

5/5 [==============================] - 0s 5ms/step - loss: 539.9505 - val_loss: 1382.1191
Epoch 234/10000
5/5 [==============================] - 0s 5ms/step - loss: 649.1176 - val_loss: 974.3289
Epoch 235/10000
5/5 [==============================] - 0s 5ms/step - loss: 615.9639 - val_loss: 879.7438
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 578.7269 - val_loss: 1179.0375
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 582.0931 - val_loss: 1120.8492
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 600.3574 - val_loss: 1064.5204
Epoch 239/10000
5/5 [==============================] - 0s 5ms/step - loss: 608.9382 - val_loss: 1055.3241
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 658.1074 - val_loss: 1111.8187
Epoch 241/10000
5/5 [==============================] - 0s 5ms/step - loss: 594.2588 - val_loss: 455.1496
Epoch 242/10000
5/5 [==============================] - 0s 5ms/ste

5/5 [==============================] - 0s 5ms/step - loss: 557.1807 - val_loss: 1113.0728
Epoch 311/10000
5/5 [==============================] - 0s 5ms/step - loss: 518.2212 - val_loss: 686.0107
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 674.4453 - val_loss: 895.5922
Epoch 313/10000
5/5 [==============================] - 0s 5ms/step - loss: 619.4511 - val_loss: 961.7487
Epoch 314/10000
5/5 [==============================] - 0s 5ms/step - loss: 584.0399 - val_loss: 1228.3269
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 675.6328 - val_loss: 707.6927
Epoch 316/10000
5/5 [==============================] - 0s 5ms/step - loss: 715.1331 - val_loss: 915.5080
Epoch 317/10000
5/5 [==============================] - 0s 6ms/step - loss: 618.5643 - val_loss: 867.6795
Epoch 318/10000
5/5 [==============================] - 0s 5ms/step - loss: 488.3541 - val_loss: 912.6249
Epoch 319/10000
5/5 [==============================] - 0s 6ms/step - 

...

Epoch 929/10000
5/5 [==============================] - 0s 5ms/step - loss: 500.9847 - val_loss: 1310.8259
Epoch 930/10000
5/5 [==============================] - 0s 5ms/step - loss: 521.0410 - val_loss: 727.0934
Epoch 931/10000
5/5 [==============================] - 0s 5ms/step - loss: 474.2779 - val_loss: 944.6254
Epoch 932/10000
5/5 [==============================] - 0s 5ms/step - loss: 509.3203 - val_loss: 990.5581
Epoch 933/10000
5/5 [==============================] - 0s 5ms/step - loss: 442.7108 - val_loss: 875.2349
Epoch 934/10000
5/5 [==============================] - 0s 5ms/step - loss: 511.5152 - val_loss: 985.6372
Epoch 935/10000
5/5 [==============================] - 0s 5ms/step - loss: 623.3780 - val_loss: 776.7415
Epoch 936/10000
5/5 [==============================] - 0s 5ms/step - loss: 785.1431 - val_loss: 829.0397
Epoch 937/10000
5/5 [==============================] - 0s 6ms/step - loss: 741.8402 - val_loss: 900.1965
Epoch 938/10000
5/5 [==============================] -

5/5 [==============================] - 0s 7ms/step - loss: 500.2196 - val_loss: 932.0776
Epoch 1007/10000
5/5 [==============================] - 0s 6ms/step - loss: 489.4536 - val_loss: 1390.7094
Epoch 1008/10000
5/5 [==============================] - 0s 6ms/step - loss: 515.0904 - val_loss: 1419.5919
Epoch 1009/10000
5/5 [==============================] - 0s 5ms/step - loss: 588.8121 - val_loss: 1318.6011
Epoch 1010/10000
5/5 [==============================] - 0s 5ms/step - loss: 768.4432 - val_loss: 1171.0454
Epoch 1011/10000
5/5 [==============================] - 0s 5ms/step - loss: 647.7180 - val_loss: 1019.5637
Epoch 1012/10000
5/5 [==============================] - 0s 5ms/step - loss: 692.2153 - val_loss: 966.0377
Epoch 1013/10000
5/5 [==============================] - 0s 5ms/step - loss: 754.2255 - val_loss: 834.5057
Epoch 1014/10000
5/5 [==============================] - 0s 5ms/step - loss: 636.9559 - val_loss: 889.5638
Epoch 1015/10000
5/5 [==============================] - 0s

5/5 [==============================] - 0s 5ms/step - loss: 714.4808 - val_loss: 1267.5438
Epoch 1084/10000
5/5 [==============================] - 0s 6ms/step - loss: 863.3065 - val_loss: 1183.6096
Epoch 1085/10000
5/5 [==============================] - 0s 6ms/step - loss: 821.0215 - val_loss: 1579.4722
Epoch 1086/10000
5/5 [==============================] - 0s 7ms/step - loss: 848.5527 - val_loss: 1021.6116
Epoch 1087/10000
5/5 [==============================] - 0s 5ms/step - loss: 904.8778 - val_loss: 1673.9808
Epoch 1088/10000
5/5 [==============================] - 0s 5ms/step - loss: 853.9161 - val_loss: 1130.1882
Epoch 1089/10000
5/5 [==============================] - 0s 6ms/step - loss: 783.9357 - val_loss: 1304.4878
Epoch 1090/10000
5/5 [==============================] - 0s 6ms/step - loss: 799.6238 - val_loss: 1488.9207
Epoch 1091/10000
5/5 [==============================] - 0s 6ms/step - loss: 797.1768 - val_loss: 1231.0822
Epoch 1092/10000
5/5 [==============================] 

Epoch 1160/10000
5/5 [==============================] - 0s 6ms/step - loss: 689.4800 - val_loss: 934.9049
Epoch 1161/10000
5/5 [==============================] - 0s 6ms/step - loss: 513.6436 - val_loss: 1627.0942
Epoch 1162/10000
5/5 [==============================] - 0s 6ms/step - loss: 586.0114 - val_loss: 1590.6320
Epoch 1163/10000
5/5 [==============================] - 0s 6ms/step - loss: 571.3187 - val_loss: 1177.6497
Epoch 1164/10000
5/5 [==============================] - 0s 6ms/step - loss: 561.4370 - val_loss: 842.3785
Epoch 1165/10000
5/5 [==============================] - 0s 5ms/step - loss: 602.0872 - val_loss: 1105.3777
Epoch 1166/10000
5/5 [==============================] - 0s 6ms/step - loss: 689.8775 - val_loss: 1138.1012
Epoch 1167/10000
5/5 [==============================] - 0s 5ms/step - loss: 550.2679 - val_loss: 1177.1516
Epoch 1168/10000
5/5 [==============================] - 0s 5ms/step - loss: 564.0504 - val_loss: 1028.2202
Epoch 1169/10000
5/5 [=================

Epoch 1237/10000
5/5 [==============================] - 0s 6ms/step - loss: 609.2137 - val_loss: 1248.2114
Epoch 1238/10000
5/5 [==============================] - 0s 5ms/step - loss: 746.3013 - val_loss: 1190.8461
Epoch 1239/10000
5/5 [==============================] - 0s 6ms/step - loss: 603.6655 - val_loss: 1036.4572
Epoch 1240/10000
5/5 [==============================] - 0s 5ms/step - loss: 506.4192 - val_loss: 1092.4352
Epoch 1241/10000
5/5 [==============================] - 0s 6ms/step - loss: 504.7831 - val_loss: 1155.1891
Epoch 1242/10000
5/5 [==============================] - 0s 6ms/step - loss: 504.5628 - val_loss: 999.8126
Epoch 1243/10000
5/5 [==============================] - 0s 5ms/step - loss: 440.7171 - val_loss: 921.9292
Epoch 1244/10000
5/5 [==============================] - 0s 5ms/step - loss: 647.3358 - val_loss: 2154.2290
Epoch 1245/10000
5/5 [==============================] - 0s 7ms/step - loss: 835.5169 - val_loss: 1424.5056
Epoch 1246/10000
5/5 [=================

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086426,313.086395,313.085632,312.241791,266.991028,266.982117,266.795868,303.509491,312.49646
Target,354.558,360.106,350.123,288.254,301.687,268.901,274.431,298.219,328.955,335.039,260.723,366.929,353.426,344.937,302.082,294.467,336.029,278.117,323.005,291.453,300.203,317.058,314.142,301.878,358.797,356.169,316.188,346.214,323.401,238.466
Error,41.471588,47.019562,37.03656,24.832428,11.399414,44.185425,38.655426,14.867432,15.868561,21.952576,52.363434,53.84256,40.339569,31.850586,11.004425,18.619415,22.942566,34.969421,9.918579,21.633423,12.883423,3.971588,1.055603,11.207642,46.555206,89.177979,49.205872,79.418121,19.89151,74.030457


In [21]:
display(mae)
display(mape)

32.73901

0.104391016

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[3757.037]] - Target[3787.925]| =  Error: [[30.88794]]; MAPE:[[0.00815432]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[3757.0364]] - Target[3756.797]| =  Error: [[0.23925781]]; MAPE:[[6.3686646e-05]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[1729.0167]] - Target[1939.2350000000001]| =  Error: [[210.21826]]; MAPE:[[0.10840268]]


[array([[30.88794]], dtype=float32),
 array([[0.23925781]], dtype=float32),
 array([[210.21826]], dtype=float32)]

80.44849

0.03887356